In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql import types as T

import json
import numpy as np
#from pyspark.ml.feature import VectorAssembler
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer
#from pyspark.mllib.evaluation import RegressionMetrics

In [2]:
def rename_columns(df, list_of_tuples):
    for (old_col, new_col) in list_of_tuples:
        df = df.withColumnRenamed(old_col, new_col)
    return df

def read_file(filepath, sqlContext):
    data_frame = sqlContext.read.csv(filepath, header=False,
                                     inferSchema=True,nullValue="-")
    while len(data_frame.columns) < 16:
        col_name = "_c" + str(len(data_frame.columns))
        data_frame = data_frame.withColumn(col_name, F.lit(None))

    data_frame = rename_columns(
        data_frame,
        [
            ("_c0", "route"),
            ("_c1", "tripNum"),
            ("_c2", "shapeId"),
            ("_c3", "shapeSequence"),
            ("_c4", "shapeLat"),
            ("_c5", "shapeLon"),
            ("_c6", "distanceTraveledShape"),
            ("_c7", "busCode"),
            ("_c8", "gpsPointId"),
            ("_c9", "gpsLat"),
            ("_c10", "gpsLon"),
            ("_c11", "distanceToShapePoint"),
            ("_c12", "timestamp"),
            ("_c13", "busStopId"),
            ("_c14", "problem"),
            ("_c15", "numPassengers")
        ]
    )

    date = "-".join(filepath.split("/")[-2].split("_")[:3])

    data_frame = data_frame.withColumn("date", F.lit(date))

    return data_frame

### Read Input Data

In [3]:
sc = SparkContext.getOrCreate()
sqlContext = pyspark.SQLContext(sc)

In [5]:
trips_data = read_file('/local/tarciso/sample-data/bulma-output/2017_05_11_veiculos.csv/part-00000', sqlContext)

In [6]:
trips_data.head(3)

[Row(route=u'372', tripNum=4, shapeId=1891, shapeSequence=6136916, shapeLat=-25.432724990605614, shapeLon=-49.27218701780396, distanceTraveledShape=10149.88, busCode=u'CC170', gpsPointId=None, gpsLat=None, gpsLon=None, distanceToShapePoint=None, timestamp=u'12:13:43', busStopId=None, problem=u'BETWEEN', numPassengers=None, date=u'2017-05-11'),
 Row(route=u'372', tripNum=4, shapeId=1891, shapeSequence=6136915, shapeLat=-25.432470122014415, shapeLon=-49.27231020615159, distanceTraveledShape=10119.046, busCode=u'CC170', gpsPointId=None, gpsLat=None, gpsLon=None, distanceToShapePoint=None, timestamp=u'12:45:14', busStopId=None, problem=u'BETWEEN', numPassengers=None, date=u'2017-05-11'),
 Row(route=u'372', tripNum=1, shapeId=1891, shapeSequence=6136650, shapeLat=-25.413362156585787, shapeLon=-49.20592429766663, distanceTraveledShape=79.983, busCode=u'CC170', gpsPointId=None, gpsLat=-25.413378, gpsLon=-49.205836, distanceToShapePoint=9.041484, timestamp=u'05:41:14', busStopId=None, problem=

In [7]:
trips_data.printSchema()

root
 |-- route: string (nullable = true)
 |-- tripNum: integer (nullable = true)
 |-- shapeId: integer (nullable = true)
 |-- shapeSequence: integer (nullable = true)
 |-- shapeLat: double (nullable = true)
 |-- shapeLon: double (nullable = true)
 |-- distanceTraveledShape: double (nullable = true)
 |-- busCode: string (nullable = true)
 |-- gpsPointId: string (nullable = true)
 |-- gpsLat: double (nullable = true)
 |-- gpsLon: double (nullable = true)
 |-- distanceToShapePoint: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- busStopId: integer (nullable = true)
 |-- problem: string (nullable = true)
 |-- numPassengers: integer (nullable = true)
 |-- date: string (nullable = false)



In [73]:
filepath = '/local/tarciso/sample-data/ticketing-sample/doc1-2017051115.txt'
ticketing_data = sqlContext.read.json(filepath)

In [75]:
#Renaming columns to english
ticketing_data = ticketing_data.select(F.col("CODLINHA").alias("route"),
                                       F.col("CODVEICULO").alias("busCode"),
                                       F.col("DATANASCIMENTO").alias("userBirthdate"),
                                       F.col("DATAUTILIZACAO").alias("cardTimestamp"),
                                       F.col("NOMELINHA").alias("lineName"),
                                       F.col("NUMEROCARTAO").alias("cardNum"),
                                       F.col("SEXO").alias("gender"))

In [76]:
ticketing_data.printSchema()

root
 |-- route: string (nullable = true)
 |-- busCode: string (nullable = true)
 |-- userBirthdate: string (nullable = true)
 |-- cardTimestamp: string (nullable = true)
 |-- lineName: string (nullable = true)
 |-- cardNum: string (nullable = true)
 |-- gender: string (nullable = true)



In [77]:
ticketing_data.head(5)

[Row(route=u'021', busCode=u'08046', userBirthdate=u'26/01/72', cardTimestamp=u'10/05/17 20:15:16,000000', lineName=u'INTERB II ANTI H', cardNum=u'0001937533', gender=u'F'),
 Row(route=u'021', busCode=u'08027', userBirthdate=u'26/01/72', cardTimestamp=u'10/05/17 13:10:24,000000', lineName=u'INTERB II ANTI H', cardNum=u'0001937533', gender=u'F'),
 Row(route=u'623', busCode=u'HA022', userBirthdate=u'06/03/71', cardTimestamp=u'10/05/17 08:23:45,000000', lineName=u'PQ.INDUSTRIAL', cardNum=u'0001311020', gender=u'F'),
 Row(route=u'000', busCode=u'03023', userBirthdate=u'06/03/71', cardTimestamp=u'10/05/17 11:54:19,000000', lineName=u'OPER S/LINHA', cardNum=u'0001311020', gender=u'F'),
 Row(route=u'TPH', busCode=u'03019', userBirthdate=u'23/11/79', cardTimestamp=u'10/05/17 13:30:10,000000', lineName=u'TERMINAL PINHEIRINHO', cardNum=u'0002425635', gender=u'F')]

## Pre-processing data

In [12]:
trips_data = trips_data.withColumn("gps_timestamp", F.concat(F.col("date"), F.lit(" "), F.col("timestamp")))
trips_data = trips_data.withColumn("gps_timestamp", F.unix_timestamp(F.col("gps_timestamp"), "YYYY-MM-DD HH:mm:ss"))

In [13]:
trips_data.select(["date","timestamp","gps_timestamp"]).show()

+----------+---------+-------------+
|      date|timestamp|gps_timestamp|
+----------+---------+-------------+
|2017-05-11| 12:13:43|   1483283623|
|2017-05-11| 12:45:14|   1483285514|
|2017-05-11| 05:41:14|   1483260074|
|2017-05-11| 05:41:16|   1483260076|
|2017-05-11| 05:41:20|   1483260080|
|2017-05-11| 05:41:27|   1483260087|
|2017-05-11| 05:41:31|   1483260091|
|2017-05-11| 05:41:33|   1483260093|
|2017-05-11| 05:41:37|   1483260097|
|2017-05-11| 05:41:45|   1483260105|
|2017-05-11| 05:41:47|   1483260107|
|2017-05-11| 05:41:52|   1483260112|
|2017-05-11| 05:42:00|   1483260120|
|2017-05-11| 05:42:03|   1483260123|
|2017-05-11| 05:42:08|   1483260128|
|2017-05-11| 05:42:12|   1483260132|
|2017-05-11| 05:42:17|   1483260137|
|2017-05-11| 05:42:21|   1483260141|
|2017-05-11| 05:42:23|   1483260143|
|2017-05-11| 05:42:25|   1483260145|
+----------+---------+-------------+
only showing top 20 rows



In [78]:
ticketing_data = ticketing_data.withColumn("card_timestamp", F.unix_timestamp(F.col("cardTimestamp"), "DD/MM/YY HH:mm:ss"))

In [79]:
ticketing_data.select(["cardTimestamp","card_timestamp"]).show()

+--------------------+--------------+
|       cardTimestamp|card_timestamp|
+--------------------+--------------+
|10/05/17 20:15:16...|    1483312516|
|10/05/17 13:10:24...|    1483287024|
|10/05/17 08:23:45...|    1483269825|
|10/05/17 11:54:19...|    1483282459|
|10/05/17 13:30:10...|    1483288210|
|10/05/17 07:52:52...|    1483267972|
|10/05/17 18:34:06...|    1483306446|
|10/05/17 06:15:31...|    1483262131|
|10/05/17 17:57:28...|    1483304248|
|10/05/17 10:03:56...|    1483275836|
|10/05/17 13:35:56...|    1483288556|
|10/05/17 07:50:18...|    1483267818|
|10/05/17 12:26:01...|    1483284361|
|10/05/17 14:34:27...|    1483292067|
|10/05/17 14:34:30...|    1483292070|
|10/05/17 14:34:32...|    1483292072|
|10/05/17 14:02:43...|    1483290163|
|10/05/17 11:04:25...|    1483279465|
|10/05/17 06:44:08...|    1483263848|
|10/05/17 17:38:18...|    1483303098|
+--------------------+--------------+
only showing top 20 rows



In [80]:
curr_date = trips_data.sample(False, 0.1, seed=0).limit(1).withColumn("curr_date", F.unix_timestamp(F.col("date"), "YYYY-MM-DD")).select("curr_date").rdd.flatMap(list).first()

In [81]:
print curr_date

1483239600


In [82]:
def get_group_N_sec(timestamp,curr_date_timestamp,N):
    secs_since_midnight = timestamp-curr_date_timestamp
    return(F.floor(secs_since_midnight/N))

In [83]:
ticketing_data = ticketing_data.withColumn("sec_group",get_group_N_sec(F.col("card_timestamp"),curr_date,30))
ticketing_data.select(["card_timestamp","sec_group"]).show()

+--------------+---------+
|card_timestamp|sec_group|
+--------------+---------+
|    1483312516|     2430|
|    1483287024|     1580|
|    1483269825|     1007|
|    1483282459|     1428|
|    1483288210|     1620|
|    1483267972|      945|
|    1483306446|     2228|
|    1483262131|      751|
|    1483304248|     2154|
|    1483275836|     1207|
|    1483288556|     1631|
|    1483267818|      940|
|    1483284361|     1492|
|    1483292067|     1748|
|    1483292070|     1749|
|    1483292072|     1749|
|    1483290163|     1685|
|    1483279465|     1328|
|    1483263848|      808|
|    1483303098|     2116|
+--------------+---------+
only showing top 20 rows



In [84]:
trips_data = trips_data.withColumn("sec_group",get_group_N_sec(F.col("gps_timestamp"),curr_date,30))
trips_data.select(["gps_timestamp","sec_group"]).show()

+-------------+---------+
|gps_timestamp|sec_group|
+-------------+---------+
|   1483283623|     1467|
|   1483285514|     1530|
|   1483260074|      682|
|   1483260076|      682|
|   1483260080|      682|
|   1483260087|      682|
|   1483260091|      683|
|   1483260093|      683|
|   1483260097|      683|
|   1483260105|      683|
|   1483260107|      683|
|   1483260112|      683|
|   1483260120|      684|
|   1483260123|      684|
|   1483260128|      684|
|   1483260132|      684|
|   1483260137|      684|
|   1483260141|      684|
|   1483260143|      684|
|   1483260145|      684|
+-------------+---------+
only showing top 20 rows



In [85]:
trips_data.count()

7900150

In [86]:
ticketing_data.count()

306906

### Removing duplicate GPS records (occurred in the same time period)

In [88]:
time_records_by_stop_time_period = trips_data.groupby(["route","busCode", "busStopId", "sec_group"]).count()
time_records_by_stop_time_period.show()

+-----+-------+---------+---------+-----+
|route|busCode|busStopId|sec_group|count|
+-----+-------+---------+---------+-----+
|  372|  CC170|    30174|      838|    1|
|  876|  BC190|    30334|     1215|    1|
|  876|  BC190|    33774|     1295|    1|
|  876|  BC190|    33740|     2123|    1|
|  876|  BC190|    34228|     2384|    1|
|  203|  GD349|    26187|     1916|    1|
|  203|  GD349|    25707|     2113|    1|
|  203|  GD349|    25385|     2174|    1|
|  511|  EA171|    28067|     1190|    1|
|  511|  EA171|    28431|     1728|    1|
|  511|  EA171|    40444|     2581|    1|
|  511|  EA171|    40533|     2589|    1|
|  307|  ML312|    26208|      864|    1|
|  307|  ML312|    25896|     2120|    1|
|  503|  GD343|    25806|     2183|    1|
|  503|  GD343|    25500|     2559|    1|
|  542|  GA134|    31562|      853|    1|
|  542|  GA134|    31259|      913|    1|
|  876|  BC189|    30273|      722|    1|
|  876|  BC189|    34246|     1941|    1|
+-----+-------+---------+---------

In [89]:
time_records_by_stop_time_period.filter(F.col("count") > 1).show()

+-----+-------+---------+---------+-----+
|route|busCode|busStopId|sec_group|count|
+-----+-------+---------+---------+-----+
|  876|  BC190|     null|      914|   14|
|  876|  BC190|     null|     1000|    3|
|  876|  BC190|     null|     1023|   21|
|  876|  BC190|     null|     1438|    5|
|  876|  BC190|     null|     1667|   15|
|  876|  BC190|     null|     1892|    6|
|  876|  BC190|     null|     2065|    9|
|  876|  BC190|     null|     2324|    8|
|  876|  BC190|     null|     2722|   18|
|  506|  EL068|     null|     1034|    8|
|  506|  EL068|     null|     2106|    4|
|  506|  EL068|     null|     2188|    6|
|  506|  EL068|     null|     2392|   14|
|  203|  HE712|     null|     1558|    2|
|  203|  HE712|     null|      715|   16|
|  203|  HE712|     null|      745|    5|
|  203|  HE712|     null|     1146|   20|
|  203|  HE712|     null|     1567|    6|
|  203|  HE712|     null|     1694|    8|
|  203|  HE712|     null|     1812|    3|
+-----+-------+---------+---------

In [90]:
window = Window.partitionBy(["route","busCode", "busStopId", "sec_group"]).orderBy(['gps_timestamp'])
filtered_trips_data = trips_data.select('*', F.rank().over(window).alias('rownum')) \
    .filter(F.col('rownum') == 1)

In [91]:
print filtered_trips_data.count()
filtered_trips_data.limit(20).toPandas()

1935516


,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,distanceToShapePoint,timestamp,busStopId,problem,numPassengers,date,gps_timestamp,sec_group,30sec_group,rownum
0,001,18,2938,6451376,-25.428310,-49.266717,50.304,BN997,None,-25.428385,...,8.811321,13:12:32,NaN,NO_PROBLEM,NaN,2017-05-11,1483287152,1585,1585,1
1,001,18,2938,6451431,-25.438079,-49.272146,2161.514,BN997,None,NaN,...,NaN,13:21:40,NaN,BETWEEN,NaN,2017-05-11,1483287700,1603,1603,1
2,001,19,2938,6451429,-25.437777,-49.271376,2077.075,BN997,None,NaN,...,NaN,13:42:00,NaN,BETWEEN,NaN,2017-05-11,1483288920,1644,1644,1
3,001,23,2938,6451422,-25.436715,-49.268677,1781.147,BN997,None,NaN,...,NaN,15:20:32,NaN,BETWEEN,NaN,2017-05-11,1483294832,1841,1841,1
4,001,28,2938,6451464,-25.435359,-49.275271,2711.088,BN997,None,NaN,...,NaN,17:15:33,NaN,BETWEEN,NaN,2017-05-11,1483301733,2071,2071,1
5,001,26,2938,6451461,-25.435749,-49.274886,2652.419,BN997,None,NaN,...,NaN,16:33:18,26166.0,BETWEEN,0.0,2017-05-11,1483299198,1986,1986,1
6,001,6,2938,6451482,-25.432037,-49.277035,3262.616,BN998,None,-25.432075,...,6.961525,09:21:06,NaN,NO_PROBLEM,NaN,2017-05-11,1483273266,1122,1122,1
7,001,7,2938,6451382,-25.427510,-49.264774,264.958,BN998,None,-25.427553,...,17.437658,09:32:30,NaN,NO_PROBLEM,NaN,2017-05-11,1483273950,1145,1145,1
8,001,9,2938,6451421,-25.436602,-49.268382,1748.936,BN998,None,NaN,...,NaN,10:29:08,NaN,BETWEEN,NaN,2017-05-11,1483277348,1258,1258,1
9,002,1,3077,4843660,-25.425285,-49.269659,8272.861,DN027,None,-25.425310,...,2.857926,07:21:07,NaN,NO_PROBLEM,NaN,2017-05-11,1483266067,882,882,1


In [93]:
filtered_trips_data = filtered_trips_data.na.drop(subset=["busStopId"])

In [94]:
filtered_trips_data.count()

559862

In [95]:
user_boardings = ticketing_data.join(filtered_trips_data, ['route','busCode','sec_group'], 'inner')

In [96]:
user_boardings.printSchema()

root
 |-- route: string (nullable = true)
 |-- busCode: string (nullable = true)
 |-- sec_group: long (nullable = true)
 |-- userBirthdate: string (nullable = true)
 |-- cardTimestamp: string (nullable = true)
 |-- lineName: string (nullable = true)
 |-- cardNum: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- card_timestamp: long (nullable = true)
 |-- tripNum: integer (nullable = true)
 |-- shapeId: integer (nullable = true)
 |-- shapeSequence: integer (nullable = true)
 |-- shapeLat: double (nullable = true)
 |-- shapeLon: double (nullable = true)
 |-- distanceTraveledShape: double (nullable = true)
 |-- gpsPointId: string (nullable = true)
 |-- gpsLat: double (nullable = true)
 |-- gpsLon: double (nullable = true)
 |-- distanceToShapePoint: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- busStopId: integer (nullable = true)
 |-- problem: string (nullable = true)
 |-- numPassengers: integer (nullable = true)
 |-- date: string (nullable = fa

In [97]:
user_boardings.select(['route','busCode','sec_group']).limit(20).toPandas()

,route,busCode,sec_group
0,020,BB300,639
1,020,CB604,2181
2,020,CB604,2181
3,020,CB604,2181
4,020,CB604,2181
5,020,CB604,2181
6,020,CB604,2181
7,021,BB601,1677
8,021,CB697,1828
9,030,BB498,782


In [98]:
user_boardings.count()

52125

In [101]:
user_boardings.select(['route','busCode','sec_group','cardNum','cardTimestamp']).orderBy('cardNum','cardTimestamp').limit(20).toPandas()

,route,busCode,sec_group,cardNum,cardTimestamp
0,828,LA053,1496,0000229948,"10/05/17 12:28:19,000000"
1,050,LA054,1458,0000310241,"10/05/17 12:09:23,000000"
2,531,EA169,897,0000314357,"10/05/17 07:28:31,000000"
3,335,DR406,905,0000314918,"10/05/17 07:32:52,000000"
4,216,CA600,2051,0000317896,"10/05/17 17:05:57,000000"
5,002,DN029,2270,0000321169,"10/05/17 18:55:17,000000"
6,811,BA018,1975,0000321469,"10/05/17 16:27:47,000000"
7,779,JC865,710,0000321916,"10/05/17 05:55:17,000000"
8,646,HA006,1578,0000322132,"10/05/17 13:09:01,000000"
9,205,BC306,984,0000323896,"10/05/17 08:12:02,000000"


In [106]:
boarding_count = user_boardings.groupby('cardNum').count()

In [110]:
total_passengers = user_boardings.select('cardNum').distinct().count()
print "Total #Passengers:", total_passengers

Total #Passengers: 44264


In [107]:
#Filtering only users with more than one ride per day
multiple_boardings = boarding_count.filter(F.col('count') > 1)

In [111]:
passengers_mult_boardings = multiple_boardings.count()
prop_mult_boardings = 100*(passengers_mult_boardings)/total_passengers
print "Passengers with Multiple Boardings:", passengers_mult_boardings, "(", prop_mult_boardings, "%)" 
multiple_boardings.show()

Passengers with Multiple Boardings: 6784 ( 15 %)
+----------+-----+
|   cardNum|count|
+----------+-----+
|0002167105|    3|
|0003637275|    2|
|0002633338|    2|
|0003437796|    2|
|0002083421|    2|
|0003801759|    2|
|0003611815|    2|
|0002788327|    2|
|0002703900|    3|
|0003768761|    2|
|0003759461|    2|
|0002752794|    2|
|0003819969|    2|
|0002277554|    2|
|0003756853|    2|
|0002986469|    7|
|0003344773|    2|
|0002911856|    2|
|0003512933|    2|
|0003697174|    2|
+----------+-----+
only showing top 20 rows



In [114]:
#Taking a look at a sample:
user_boardings.filter(F.col('cardNum') == '0002167105').limit(20).toPandas()

,route,busCode,sec_group,userBirthdate,cardTimestamp,lineName,cardNum,gender,card_timestamp,tripNum,...,gpsLon,distanceToShapePoint,timestamp,busStopId,problem,numPassengers,date,gps_timestamp,30sec_group,rownum
0,030,GR123,1409,07/08/91,"10/05/17 11:44:57,000000",INTERBAIRROS III,0002167105,M,1483281897,2,...,-49.267173,50.379257,11:44:50,32611,NO_PROBLEM,2,2017-05-11,1483281890,1409,1
1,030,GR123,1409,07/08/91,"10/05/17 11:44:57,000000",INTERBAIRROS III,0002167105,M,1483281897,2,...,-49.265710,75.325790,11:44:32,32848,NO_PROBLEM,0,2017-05-11,1483281872,1409,1
2,030,KB699,1055,07/08/91,"10/05/17 08:47:49,000000",INTERBAIRROS III,0002167105,M,1483271269,2,...,-49.262606,20.480734,08:47:35,32607,TRIP_PROBLEM,1,2017-05-11,1483271255,1055,1
